# 1. ETL ➡ Extraer, Transformar y Cargar

Obtengo los datos a través del cliente que está interesado en que le analicen los datos de su negocio. En este caso, es una farmacia comunitaria de Canarias, en Tenerife.
 
El cliente ha obtenido los datos a partir del software de su negocio, llamado [Unycop](https://www.unycop.es/). Los datos se encuentran en formato excel.

In [15]:
#%pip install xlrd>=2.0.1
#%pip install sqlalchemy
#%pip install xlwt
#%pip install mysql-connector-python
#%pip install SQLAlchemy==2.0.10
#%pip install pymysql
#%pip install mysql

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import os
import numpy as np 
import json
import warnings
warnings.filterwarnings('ignore')
import sys
sys.path.append('../src')

import mysql
import configparser
import mysql.connector as conn
import pymysql
from sqlalchemy import create_engine

import importlib
import funciones
importlib.reload(funciones) 
from funciones import ExcelManager

### Limpieza del archivo excel articulos

In [18]:
df = pd.read_excel('../data/data_raw/articulos.xlsx')
df = df.iloc[:, 1:]
df.columns = df.iloc[0]
df = df.iloc[1:, :]
df = df.reset_index(drop=True)


In [19]:
df.head()

,Código,Nombre,Familia,Mínimo,Existencias,P.V.P.,NaN,Ventas,Porcent.
0,702686,JARDIANCE 10 MG 30 COMP,Financiables,4,5,"49,53",NaN,"18.188,36","0,80%"
1,706414,TOUJEO 300 UI SOL INYECT 3 PLUMAS 1.5 M,Financiables,5,6,"48,68",NaN,"18.011,60","0,80%"
2,700588,FORXIGA 10 MG 28 COMP,Financiables,3,3,"44,25",NaN,"16.149,68","0,71%"
3,704632,TRULICITY 1.5 MG SOL INY 4 PLUMAS PREC,Financiables,2,2,"123,22",NaN,"15.895,38","0,70%"
4,935353,NEXIUM MUPS 40 MG 28 COMP,Financiables,5,4,"24,14",NaN,"14.966,80","0,66%"


Borro la columna vacía.

In [21]:
df = df.drop_duplicates()
df = df.rename(columns={df.columns[6]: 'columna_vacia'})
columna_vacia = 'columna_vacia'
df = df.drop(columns=[columna_vacia], errors='ignore')
df = df.dropna()

In [24]:
df.head()

,Código,Nombre,Familia,Mínimo,Existencias,P.V.P.,Ventas,Porcent.
0,702686,JARDIANCE 10 MG 30 COMP,Financiables,4,5,"49,53","18.188,36","0,80%"
1,706414,TOUJEO 300 UI SOL INYECT 3 PLUMAS 1.5 M,Financiables,5,6,"48,68","18.011,60","0,80%"
2,700588,FORXIGA 10 MG 28 COMP,Financiables,3,3,"44,25","16.149,68","0,71%"
3,704632,TRULICITY 1.5 MG SOL INY 4 PLUMAS PREC,Financiables,2,2,"123,22","15.895,38","0,70%"
4,935353,NEXIUM MUPS 40 MG 28 COMP,Financiables,5,4,"24,14","14.966,80","0,66%"


Limpieza de espacios en blanco al pasar el archivo pdf a excel. Algunas filas se quedaban nulas. 

In [25]:
columnas_a = ['Código', 'Nombre', 'Familia', 'Mínimo', 'Existencias', 'P.V.P.', 'Ventas', 'Porcent.']
eliminar = ['Código', 'Nombre', 'Familia', 'Mínimo', 'Existencias', 'P.V.P.', 'Ventas', 'Porcent.']

for columna, valor in zip(columnas_a, eliminar):
    df = df[~(df[columna] == valor)]

In [26]:
df.to_excel('../data/data_clean/archivos.xlsx', index=False)

### Limpieza de los 72 archivos excel lineas_ventas 

Primero muestro un ejemplo de cómo se encontraba inicialmente el archivo excel: lineas_ventas_diciembre_17.

In [30]:
dic_17 = pd.read_excel('../data/data_raw/2017/lineas_ventas_diciembre_17.xls')

In [31]:
dic_17.head(5)

,Unnamed: 0,Fecha,Hora,Tipo de Operación,Empresa,Código,Denominación,Organismo,Cantidad (Unidades),Pvp,PVP Facturado,Importe Bruto,Descuento,Importe Neto,Cliente,Perfil de Cliente,Vendedor,Aporta.Subv.,Número de Puesto,Existencias Anteriores,Existencias Posteriores
0,NaN,01/12/17,08:10,Contado,1,770065,ORFIDAL 1 MG 50 COMPRIMIDOS,091 - TSI 002,1,1.65,1.65,0.17,0.0,0.17,NaN,NaN,ADA,0,1,36,35
1,NaN,01/12/17,08:10,Contado,1,770065,ORFIDAL 1 MG 50 COMPRIMIDOS,095 - RP RE DOE,1,1.65,1.65,0.00,0.0,0.00,NaN,NaN,ADA,0,1,37,36
2,NaN,01/12/17,08:10,Contado,1,695435,VIBRACINA 100 MG 14 CAPSULAS,123 - TSI 001 *,1,2.40,2.40,0.00,0.0,0.00,NaN,NaN,ADA,0,1,2,1
3,NaN,01/12/17,08:17,Contado,1,656578,MICARDIS 80 MG 28 COMP,002 - R.G. S.S.,-1,19.20,19.20,-1.92,0.0,-1.92,NaN,NaN,ADA,0,3,21,22
4,NaN,01/12/17,08:17,Contado,1,656578,MICARDIS 80 MG 28 COMP,094 - RG RE DOE,1,19.20,19.20,1.92,0.0,1.92,NaN,NaN,ADA,0,3,22,21


Ahora se extraen y se filtran los excels según el tipo de archivo. En este caso solo se acaban filtrando los archivos lineas_ventas.

In [ ]:
excelManager = ExcelManager()
carpeta_origen = r"C:\Users\blanx\ironhack\Final_proyect_SME_real_case\Final_proyect_SME_real_case\data\data_raw"
excels  = excelManager.extraerExcels(carpeta_origen)
excelManager.filtrarExcels(excels)

Dentro de la función de clase .filtrarExcels(), la función .lineasVentasToDataFrame() limpia el archivo excel lineas_ventas según las necesidades especificas de la tabla.  

Luego para mi conocimiento acabo guardando los excels limpios en la carpeta data_clean. Aunque estos archivos no los acabo utilizando. 

In [ ]:
carpeta_destino = r"C:\Users\blanx\ironhack\Final_proyect_SME_real_case\Final_proyect_SME_real_case\data\data_clean"
excelManager.exportarDfsToExcels(carpeta_destino)

## Resumen

Primero limpio el archivo pdf articulos para volverlo un excel y poder obtener las columnas Familia y Mínimo. Luego creo una clase llamada ExcelManager para limpiar los excels, según el nombre del archivo y el tipo de datos que contiene internamente (en concreto lineas_ventas). También guarda los excels limpios y contiene otras funciones que aplico en el siguiente apartado.


# 2. Base de datos en MySQL

### Conexión a MySQL
Función para crear el engine de conexión de la base de datos.


In [ ]:
excelManager.conectarMySQL()

### Creación de la base de datos ➡ Formación de dataframes y cada columna de valores
Función para crear la base de datos.


In [ ]:
excelManager.crearBDMySQL()

### Calcular las tablas en base a los Data Frames extraídos 

Función para filtrar y guardar los dataframes en un diccionario, según el tipo de data_frames: "lineas_ventas", "facturacion_familia" o "facturacion".


In [ ]:
data_frames_sql = excelManager.obtenerDFstoMySQL()


### Carga de base de datos
Función para subir los dataframes con toda la información de la base de datos.


In [ ]:
excelManager.volcarDatosMySQL(data_frames_sql) 


### Entity Relationship Diagram - ERD

ERD creado en MySQL

![bd](../images/ERD.jpg)

### Creación de uniones de los diagramas

```SQL
-- MySQL Workbench Forward Engineering

SET @OLD_UNIQUE_CHECKS=@@UNIQUE_CHECKS, UNIQUE_CHECKS=0;
SET @OLD_FOREIGN_KEY_CHECKS=@@FOREIGN_KEY_CHECKS, FOREIGN_KEY_CHECKS=0;
SET @OLD_SQL_MODE=@@SQL_MODE, SQL_MODE='ONLY_FULL_GROUP_BY,STRICT_TRANS_TABLES,NO_ZERO_IN_DATE,NO_ZERO_DATE,ERROR_FOR_DIVISION_BY_ZERO,NO_ENGINE_SUBSTITUTION';

-- -----------------------------------------------------
-- Schema mydb
-- -----------------------------------------------------
-- -----------------------------------------------------
-- Schema farmacia
-- -----------------------------------------------------
-- -----------------------------------------------------
-- Schema farmacia
-- -----------------------------------------------------
CREATE SCHEMA IF NOT EXISTS `farmacia` DEFAULT CHARACTER SET utf8mb4 COLLATE utf8mb4_0900_ai_ci ;
USE `farmacia` ;

-- -----------------------------------------------------
-- Table `farmacia`.`empleados`
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS `farmacia`.`empleados` (
  `index` BIGINT NOT NULL,
  `Vendedor` TEXT NULL DEFAULT NULL,
  INDEX `ix_Empleados_index` (`index` ASC) VISIBLE,
  PRIMARY KEY (`index`))
ENGINE = InnoDB
DEFAULT CHARACTER SET = utf8mb4
COLLATE = utf8mb4_0900_ai_ci;


-- -----------------------------------------------------
-- Table `farmacia`.`medicamentos`
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS `farmacia`.`medicamentos` (
  `index` BIGINT NOT NULL,
  `Codigo` BIGINT NULL DEFAULT NULL,
  `Denominacion` TEXT NULL DEFAULT NULL,
  `Pvp` DOUBLE NULL DEFAULT NULL,
  INDEX `ix_Medicamentos_index` (`index` ASC) VISIBLE,
  PRIMARY KEY (`index`))
ENGINE = InnoDB
DEFAULT CHARACTER SET = utf8mb4
COLLATE = utf8mb4_0900_ai_ci;


-- -----------------------------------------------------
-- Table `farmacia`.`ventas`
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS `farmacia`.`ventas` (
  `index` BIGINT NOT NULL,
  `Fecha` TEXT NULL DEFAULT NULL,
  `Hora` TEXT NULL DEFAULT NULL,
  `Organismo` TEXT NULL DEFAULT NULL,
  `Cantidad` BIGINT NULL DEFAULT NULL,
  `Pvp_facturado` DOUBLE NULL DEFAULT NULL,
  `Importe_bruto` DOUBLE NULL DEFAULT NULL,
  `Descuento` DOUBLE NULL DEFAULT NULL,
  `Importe_neto` DOUBLE NULL DEFAULT NULL,
  `Puesto` BIGINT NULL DEFAULT NULL,
  `Existencias` BIGINT NULL DEFAULT NULL,
  `Ticket` DOUBLE NULL DEFAULT NULL,
  `Familia` TEXT NULL DEFAULT NULL,
  `Mínimo` DOUBLE NULL,
  `medicamentos_index` BIGINT NOT NULL,
  `empleados_index` BIGINT NOT NULL,
  INDEX `ix_Ventas_index` (`index` ASC) VISIBLE,
  PRIMARY KEY (`index`),
  INDEX `fk_ventas_medicamentos_idx` (`medicamentos_index` ASC) VISIBLE,
  INDEX `fk_ventas_empleados1_idx` (`empleados_index` ASC) VISIBLE,
  CONSTRAINT `fk_ventas_medicamentos`
    FOREIGN KEY (`medicamentos_index`)
    REFERENCES `farmacia`.`medicamentos` (`index`)
    ON DELETE NO ACTION
    ON UPDATE NO ACTION,
  CONSTRAINT `fk_ventas_empleados1`
    FOREIGN KEY (`empleados_index`)
    REFERENCES `farmacia`.`empleados` (`index`)
    ON DELETE NO ACTION
    ON UPDATE NO ACTION)
ENGINE = InnoDB
DEFAULT CHARACTER SET = utf8mb4
COLLATE = utf8mb4_0900_ai_ci;


SET SQL_MODE=@OLD_SQL_MODE;
SET FOREIGN_KEY_CHECKS=@OLD_FOREIGN_KEY_CHECKS;
SET UNIQUE_CHECKS=@OLD_UNIQUE_CHECKS;
```

### Queries
Compruebo que la tabla se ha subido adecuadamente y que el número de filas coincide con el original y que la base de datos no está rota.

```SQL
SELECT * FROM ventas;
```
![bd](../images/MySQL1.jpg)


```SQL
SELECT COUNT(*) AS total_filas
FROM ventas;
```
![bd](../images/MySQL2.jpg)

```SQL
SELECT COUNT(DISTINCT `index`) AS total_filas_distinct
FROM ventas;
```
![bd](../images/MySQL3.jpg)

## Resumen

Vuelvo a utilizar mi querida clase ExcelManager para formar los dataframes según mis necesidades, conectarme automaticamente a MySQL, crear la base de datos denominada farmacia, calcular las tablas a partir de los Data Frames extraídos (medicamentos, empleados y ventas) y volcar los datos a MySQl.

Compruebo las relaciones entre las tablas ➡ Entity Relationship Diagram - ERD y finalmente me aseguro de que la tabla se ha subido adecuadamente. Si nos fijamos bien tenemos 1.214.357 filas!!!

# 3. DashBoards en Power BI

![pwbi](../images/powerbi1.jpg)


![pwbi](../images/powerbi2.jpg)


![pwbi](../images/powerbi3.jpg)


## Resumen

A continuación indicaré cada gráfica por orden de visualización, es decir, de izquierda a derecha y de arriba hacia abajo. 

- DashBoard 1: Intro

Foto bonita de la farmacia. MIRA QUE CUQUI!!

- DashBoard 2: Análisis de ventas

1º Facturación Total. 2º Filtro por año, mes y día de los últimos 6 años, desde el 1/12/2017 hasta el 30/11/2023. 3º Facturación en porcentaje por organismo. 4º Facturación por años y meses. 5º Ventas por familia de productos y 6º Matriz que indica los nombres de los medicamentos, las cantidades compradas por clientes y la cantidad cobrada por la farmacia. 

- DashBoard 3: Análisis de tendencias y empleados

1º Descuentos por meses.  2º Filtro por año, mes y día. 3º Filtro por Empleado. 4º Ventas por empleados y años y 5º Ventas por horas.
